<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span><ul class="toc-item"><li><span><a href="#Preprocess-10X" data-toc-modified-id="Preprocess-10X-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Preprocess 10X</a></span></li><li><span><a href="#Load-10X" data-toc-modified-id="Load-10X-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load 10X</a></span></li><li><span><a href="#Reduce-dimensions" data-toc-modified-id="Reduce-dimensions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Reduce dimensions</a></span></li></ul></li><li><span><a href="#Main" data-toc-modified-id="Main-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Main</a></span><ul class="toc-item"><li><span><a href="#Load-data-and-preprocess" data-toc-modified-id="Load-data-and-preprocess-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load data and preprocess</a></span></li><li><span><a href="#Reduce-dimensions" data-toc-modified-id="Reduce-dimensions-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Reduce dimensions</a></span></li><li><span><a href="#Display-PCA" data-toc-modified-id="Display-PCA-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Display PCA</a></span></li></ul></li></ul></div>

# Imports

In [ ]:
import scprep
import matplotlib.pyplot as plt
# from functions import *

# Functions

## Preprocess 10X

In [2]:
#' This function takes a data-frame (genes x cells), 
#' creates a Scprep object with it and filters the object for default tags such as  
#' Min and max nFeature_RNA and % of MT
#' 
#'
#' @param data data-frame
#' @param percent_mt integer [0-100]
#' @param max_features integer [0-Inf]
#' @param min_features integer [0-Inf]
#' @return Preprocessed Seurat object
#' @export

def preprocess_10X(data, name='10X-project', percent_mt=20, max_features=5000, min_features=200):
    
    #Remove empty cells and empty genes
    scprep_data = scprep.filter.filter_empty_cells(data)
    scprep_data = scprep.filter.filter_empty_genes(scprep_data)
    
    #Remove elements based on mythocondrial percentage
    mt_genes = scprep.select.get_gene_set(data, starts_with=["MT-", "mt-"])
    scprep_data = scprep.filter.filter_gene_set_expression(data=scprep_data, genes=mt_genes, percentile=100-percent_mt)
    
    #Remove elements based on number of cell and number of features
    scprep_data = scprep.filter.filter_library_size(data=scprep_data, cutoff=(min_features, max_features), keep_cells='between')
    scprep_data = scprep.filter.filter_rare_genes(data=scprep_data, cutoff=0, min_cells=3)

    return scprep_data

## Load 10X

In [3]:
# This function takes the path to a 10X output folder and instanciates the Scprep object
#
# @param file string (path to file)
# @param percent_mt integer [0-100]
# @param max_features integer [0-Inf]
# @param min_features integer [0-Inf]
# @return Preprocessed Seurat object
# @export

def load_10X(dir_path, name='10X-project',percent_mt=20, max_features=5000, min_features=200):
    data = scprep.io.load_10X(dir_path, sparse=True, gene_labels='both')
    return preprocess_10X(data=data, percent_mt=percent_mt, max_features=max_features, min_features=min_features)


## Reduce dimensions

In [9]:
# This function reduces the dimensions of the Normalized Seurat object
# Runs PCA and then UMAP and then performs clustering

# @param seurat.obj S4 instance
# @param ndims integer [3-100]
# @param res double [0-3]
# @return Seurat object with reduction embeddings
# @export

def reduce_dim(data, ndims=15, res=.1):
    data = scprep.normalize.library_size_normalize(data)
#     data = scprep.transform.sqrt(data)

    return data

# Main

## Load data and preprocess

In [5]:

data_paths = [
    "../data/14_EAE_all_cells.MPS12344479-A02.sorted.1229.merged.10x_outputs/filtered_feature_bc_matrix",
    "../data/15_EAE_all_cells.MPS12344479-B02.sorted.1229.merged.10x_outputs/filtered_feature_bc_matrix",
    "../data/16_WT_all_cells.MPS12344479-H01.sorted.1229.merged.10x_outputs/filtered_feature_bc_matrix"
]

# seurat_path = "../../tutorials/seurat_guidedClustering/seurat_clusteringTutorial/filtered_gene_bc_matrices/hg19"
data = load_10X(dir_path=data_paths[0], percent_mt=20, max_features=5000, min_features=200)
data.head()


,Sox17 (ENSMUSG00000025902),Mrpl15 (ENSMUSG00000033845),Lypla1 (ENSMUSG00000025903),Tcea1 (ENSMUSG00000033813),Rgs20 (ENSMUSG00000002459),Atp6v1h (ENSMUSG00000033793),Rb1cc1 (ENSMUSG00000025907),4732440D04Rik (ENSMUSG00000090031),St18 (ENSMUSG00000033740),Pcmtd1 (ENSMUSG00000051285),...,Vamp7 (ENSMUSG00000051412),Tmlhe (ENSMUSG00000079834),AC133103.1 (ENSMUSG00000079190),Csprs (ENSMUSG00000062783),AC125149.3 (ENSMUSG00000079800),AC168977.2 (ENSMUSG00000094915),AC168977.1 (ENSMUSG00000079808),AC149090.1 (ENSMUSG00000095041),CAAA01118383.1 (ENSMUSG00000063897),CAAA01147332.1 (ENSMUSG00000095742)
0,,,,,,,,,,,,,,,,,,,,,
AAACCTGGTATCGCAT-1,3.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AAACGGGAGAACTGTA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AAACGGGGTTCTCATT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
AAACGGGGTTCTGAAC-1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
AAAGATGCAATGACCT-1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


## Reduce dimensions

In [10]:
reduce_dim(data)

,Sox17 (ENSMUSG00000025902),Mrpl15 (ENSMUSG00000033845),Lypla1 (ENSMUSG00000025903),Tcea1 (ENSMUSG00000033813),Rgs20 (ENSMUSG00000002459),Atp6v1h (ENSMUSG00000033793),Rb1cc1 (ENSMUSG00000025907),4732440D04Rik (ENSMUSG00000090031),St18 (ENSMUSG00000033740),Pcmtd1 (ENSMUSG00000051285),...,Vamp7 (ENSMUSG00000051412),Tmlhe (ENSMUSG00000079834),AC133103.1 (ENSMUSG00000079190),Csprs (ENSMUSG00000062783),AC125149.3 (ENSMUSG00000079800),AC168977.2 (ENSMUSG00000094915),AC168977.1 (ENSMUSG00000079808),AC149090.1 (ENSMUSG00000095041),CAAA01118383.1 (ENSMUSG00000063897),CAAA01147332.1 (ENSMUSG00000095742)
0,,,,,,,,,,,,,,,,,,,,,
AAACCTGGTATCGCAT-1,12.195122,0.000000,0.000000,4.065041,0.0,0.000000,8.130081,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,4.065041
AAACGGGAGAACTGTA-1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,8.665511,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,8.665511
AAACGGGGTTCTCATT-1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,5.065856,2.532928,0.000000
AAACGGGGTTCTGAAC-1,0.000000,0.000000,0.000000,2.319109,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.319109,4.638219
AAAGATGCAATGACCT-1,0.000000,2.428363,2.428363,0.000000,0.0,2.428363,0.000000,0.0,0.0,2.428363,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,4.856727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTCAAGTACCT-1,0.000000,16.863406,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
TTTGTCAAGGGTTCCC-1,0.000000,0.000000,0.000000,0.000000,0.0,4.242681,4.242681,0.0,0.0,0.000000,...,2.121341,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.121341
TTTGTCACATCTGGTA-1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.484321


## Display PCA

In [16]:
data_pcs = scprep.reduce.pca(data, n_pca=100)

C:\Users\sburw\AppData\Roaming\Python\Python37\site-packages\scprep\reduce.py:289: FutureWarning: n_pca is deprecated. Setting n_components=100.
  "n_pca is deprecated. Setting n_components={}.".format(n_pca), FutureWarning


In [28]:
# fig, ax = plt.subplots(1, figsize=(5,5))

print(data_pcs[:, 1])
# # ax.scatter(data_pcs[:,0], data_pcs[:,1], s=1)
# ax.set_xlabel('PC1')
# ax.set_ylabel('PC2')
# ax.set_title('T-cell - PCA')

# fig.tight_layout()
data_pcs

TypeError: '(slice(None, None, None), 1)' is an invalid key